In [1]:
import requests
import re
import os
import time

from urllib import request
from bs4 import BeautifulSoup

import glob
import re
from gensim.models import word2vec

import math

/usr/local/lib/python3.8/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def nocScraping(novelID, limit=0):
   
   #set UA
   header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:62.0) Gecko/20100101 Firefox/62.0"}
   
   #set cookie
   cookie = {"over18": "yes"}
   
   #親URLをセット
   chapterlistUrl = "https://novel18.syosetu.com/" + str(novelID) + "/"
   responseCL  = requests.get(url=chapterlistUrl, headers=header, cookies=cookie)
   chapterlistHtml = responseCL.content
   soupCL = BeautifulSoup(chapterlistHtml, "html.parser")
   sublist = soupCL.find_all("dl", attrs={"class","novel_sublist2"})
   #print(len(sublist))
   
   
   novelDirPass = "data/" + novelID
   count = 1
   
   if(limit == 0):
       countlimit = len(sublist) + 1
   else:
       countlimit = limit + 1
   
   while(count < countlimit):
       #set url
       url = "https://novel18.syosetu.com/" + str(novelID) + "/" + str(count) + "/"

       #get html
       response  = requests.get(url=url, headers=header, cookies=cookie)
       html = response.content

       #set BeautifulSoup
       soup = BeautifulSoup(html, "html.parser")

       #scraping
       mainText = soup.find("div", attrs={"id":"novel_honbun"})
       try:
           mainTextLines = mainText.find_all("p")
       except AttributeError:
           pass

       allText = ""

       for mainTextLine in mainTextLines:
           text = str(mainTextLine)
           text = re.sub("<p.*\">","",text)
           text = re.sub("</p>","",text)
           text = text.replace("<br/>","")
           if(text != ""):
               allText = allText + text + "\n"

       allText = re.sub("\n\n","\n",allText)
       allText = re.sub("　","",allText)
       #print(allText)
       
       os.makedirs(novelDirPass, exist_ok=True)
       newFilePass = novelDirPass + "/" + novelID + "_" + str(count) + ".txt"
       if not os.path.isfile(newFilePass):
           with open(newFilePass, mode="w", encoding="UTF-8") as f:
               f.write(allText)
       
       count = count +1
       
def getNcode(limit):
   response = requests.get("https://api.syosetu.com/novel18api/api",params={"out":"json","nocgenre":1,"sasie":0,"type":"re","ispickup":1,"lim":limit,"of":"n"})
   #print(response.json()[1]["ncode"])
   count = 1
   ncodelist=[]
   
   while(count < limit+1):
       ncodelist.append(response.json()[count]["ncode"])
       count = count + 1
   #print(ncodelist)
   return ncodelist
   

def getNovelText(number,limit=0):
   ncodelist = getNcode(number)
   count = 1
   while(count < number):
       nocScraping(ncodelist[count],limit)
       count = count + 1
       print("count = "+str(count)+" ("+str(math.floor((count/number)*100))+"%)")
       time.sleep(1.5)

In [3]:
getNcode(10) # 関数テスト

['N2697GX',
 'N2953FZ',
 'N3474GV',
 'N0417GR',
 'N2588GX',
 'N4651GX',
 'N6535GX',
 'N1114GN',
 'N6821GT',
 'N3081EN']

In [4]:
getNovelText(500,10)

count = 2 (0%)
count = 3 (0%)
count = 4 (0%)
count = 5 (1%)
count = 6 (1%)
count = 7 (1%)
count = 8 (1%)
count = 9 (1%)
count = 10 (2%)
count = 11 (2%)
count = 12 (2%)
count = 13 (2%)
count = 14 (2%)
count = 15 (3%)
count = 16 (3%)
count = 17 (3%)
count = 18 (3%)
count = 19 (3%)
count = 20 (4%)
count = 21 (4%)
count = 22 (4%)
count = 23 (4%)
count = 24 (4%)
count = 25 (5%)
count = 26 (5%)
count = 27 (5%)
count = 28 (5%)
count = 29 (5%)
count = 30 (6%)
count = 31 (6%)
count = 32 (6%)
count = 33 (6%)
count = 34 (6%)
count = 35 (7%)
count = 36 (7%)
count = 37 (7%)
count = 38 (7%)
count = 39 (7%)
count = 40 (8%)
count = 41 (8%)
count = 42 (8%)
count = 43 (8%)
count = 44 (8%)
count = 45 (9%)
count = 46 (9%)
count = 47 (9%)
count = 48 (9%)
count = 49 (9%)
count = 50 (10%)
count = 51 (10%)
count = 52 (10%)
count = 53 (10%)
count = 54 (10%)
count = 55 (11%)
count = 56 (11%)
count = 57 (11%)
count = 58 (11%)
count = 59 (11%)
count = 60 (12%)
count = 61 (12%)
count = 62 (12%)
count = 63 (12%)
co

In [5]:
word_list = []

from janome.tokenizer import Tokenizer
t = Tokenizer()
def extract_words(text):
  tokens = t.tokenize(text)
  return [token.base_form for token in tokens if token.part_of_speech.split(",")[0] in ["名詞", "動詞", "形容詞"]]

In [6]:
for path in glob.glob("./data/*/*.txt"):
  with open(path , encoding="UTF-8") as f:
    try:
      for s_line in f:
        s_line = s_line.replace("<ruby><rb>","")
        s_line = s_line.replace("</rb><rp>(</rp><rt>","")
        s_line = s_line.replace("</rt><rp>(</rp></ruby>","")
        word_list.append(extract_words(s_line))
    except UnicodeDecodeError:
      pass

In [7]:
with open("./wordlist.txt", mode="w", encoding="UTF-8") as f:
  f.write(str(word_list))

In [8]:
model = word2vec.Word2Vec(word_list , vector_size=100, min_count=5, window=5, epochs=100)
model.save("dosukebe.model")

In [9]:

model.wv.most_similar("金")

[('お金', 0.7200292348861694),
 ('手数料', 0.6027623414993286),
 ('現金', 0.5924906134605408),
 ('財産', 0.5718393325805664),
 ('貨幣', 0.5663682222366333),
 ('費用', 0.5661934018135071),
 ('日銭', 0.5569566488265991),
 ('金額', 0.5551134943962097),
 ('多額', 0.5545421838760376),
 ('潤沢', 0.5538561344146729)]